## INITIALIZE REQUIREMENTS

### Load Dependencies

In [1]:
%matplotlib inline
# !pip install python-dotenv
# load_dotenv()
# from dotenv import load_dotenv

import os, glob, warnings, datacube, rasterio, folium, json
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as rio
import pandas as pd
from rasterio.merge import merge
from rasterio.plot import show
from shapely.geometry import Point
from shapely.geometry import Polygon


from scipy.ndimage import uniform_filter
from scipy.ndimage import variance
from skimage.filters import threshold_minimum
from datacube.utils.geometry import Geometry

from deafrica_tools.spatial import xr_rasterize
from deafrica_tools.datahandling import load_ard
from deafrica_tools.plotting import display_map, rgb
from deafrica_tools.areaofinterest import define_area

from typing import Literal

from IPython.display import clear_output
from IPython.display import display

warnings.filterwarnings("ignore")

In [2]:
# local modules
from tools.gdrive import GDrive
gd = GDrive()

### Connect to the datacube

Connect to the datacube so we can access DEA data.
The `app` parameter is a unique name for the analysis which is based on the notebook file name.

In [3]:
dc = datacube.Datacube(app="Radar_water_detection")

### G-Drive Folder IDs and Timerange Definitions

In [4]:
# Create folder ids by copying the ID from the g-drive folder url
F_MN_FID = "1w_0Ib2RkdzmwvwQbxJnjBqzC7mD8kGeA"
F_MD_FID = None

PRF_MN_FID = "17yK8SoaZW_p8qJFJZS7QChbAhymA6Stm"
PRF_MD_FID = None

POF_MN_FID = None
POF_MD_FID = None

FE_MN_FID = "1N_ttkyNHG_RHqpFkpmAsCCNFy9s2_Ird"
FE_MD_FID = None

In [5]:
# Define main time period of analysis
timerange = ('2024-02', '2024-09')

# Define sub-periods of analysis - should be within main time period
pre_flood = ['2024-02', '2024-03', '2024-04'] 
flood = ['2024-05', '2024-06', '2024-07', '2024-08', '2024-09'] 
post_flood = []

# Run 1. aoi-threshold.ipynb to get the value of th_aoi and store it here.
th_aoi = -27.395682

### Filter and Classifier Functions

In [6]:
# Function to apply lee filtering on S1 image. Speckle Filter
def lee_filter(da, size):
    """
    Apply lee filter of specified window size.
    Adapted from https://stackoverflow.com/questions/39785970/speckle-lee-filter-in-python

    """
    img = da.values
    img_mean = uniform_filter(img, size)
    img_sqr_mean = uniform_filter(img**2, size)
    img_variance = img_sqr_mean - img_mean**2

    overall_variance = variance(img)

    img_weights = img_variance / (img_variance + overall_variance)
    img_output = img_mean + img_weights * (img - img_mean)

    return img_output

# Classifier Function
def S1_water_classifier(da, threshold):
    water_data_array = da < threshold
    return water_data_array.to_dataset(name="s1_water")

### Operational Funtions

In [7]:
# Generate raster outputs
def gen_output(DS:xr.DataArray, i:list, poly:Geometry, cell:int, aoi_m:list, 
               period:Literal["preflood", "flood", "postflood", "flood_extents"], 
               measure:Literal['mean', 'median']):
   
    """
    DS: Dataset to be converted into raster output
    i: index of centroid list
    poly: polygon feature used to create the DS
    cell: cell number being processed
    aoi_m: main aoi mosaic of the entire grid
    period: time period of the process - PRE_FLOOD, FLOOD
    measure: central tendency measurement of the DS - mean, median
    """

    err = []
    
    pm_dict = {
        "preflood_mean": [pre_flood, PRF_MN_FID],
        "preflood_median": [pre_flood, PRF_MD_FID],
        "flood_mean": [flood, F_MN_FID],
        "flood_median": [flood, F_MD_FID],
        "postflood_mean": [post_flood, POF_MN_FID],
        "postflood_median": [post_flood, POF_MD_FID],
        "flood_extents_mean": [timerange, FE_MN_FID],
        "flood_extents_median": [timerange, FE_MD_FID]
    }    

    poly_gdf = gpd.GeoDataFrame(geometry=[poly], crs=poly.crs)
    lat_range = (poly_gdf.total_bounds[1], poly_gdf.total_bounds[3])
    lon_range = (poly_gdf.total_bounds[0], poly_gdf.total_bounds[2])
    g = poly_gdf.centroid

    data_val = "CELL_" + str(i[2]) + "_{}_{}".format(period.upper(), measure.upper())
    data_name = data_val + ".tif"
    data_out = "output/{}/".format(period) + data_name
    DS.rio.to_raster(data_out)
    
    # preflood meta
    data_dict = {
        "GRID_CELL_ID": i[2],
        "start_time": pm_dict['{}_{}'.format(period, measure)][0][0], # pre_flood[0] 
        "end_time": pm_dict['{}_{}'.format(period, measure)][0][-1], # pre_flood[-1]
        "lat": lat_range, 
        "lon": lon_range,
        "centroid": "{}, {}".format(g.y[0], g.x[0]),
        "crs": str(poly.crs)
        }
    
    text_flie_name = data_val + "_META.json"
    data_meta_path = "output/{}/".format(period) + text_flie_name
    with open(data_meta_path, 'w') as f:
        json.dump(data_dict, f)
    
    try:
        gd.upload_files([data_out, data_meta_path], pm_dict['{}_{}'.format(period, measure)][1]) # pm_dict['preflood_mean'][1] = FOLDER_ID
    except Exception as e:
        err.append([g.x[0], g.y[0], i[2], "U-PRF"])
        print(
            "\n\n"
            + "\033[31m"
            + "ERROR UPLOADING GRID CELL ID {} NO.  {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(
                i[2], cell, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
            )
            + "\033[0m"
        )
        print("UPLOAD ERROR: {}".format(e))

    return err

In [8]:
# Iterate through the input grid
def iterate_grid(aoi_m, c):
    
    e_log = []
    cell = 1
    for aoi, i in zip(aoi_m, c):
        geopolygon = Geometry(aoi["features"][0]["geometry"], crs="epsg:4326")
        geopolygon_gdf = gpd.GeoDataFrame(geometry=[geopolygon], crs=geopolygon.crs)
        g = geopolygon_gdf.centroid
        print(
            "\n\n"
            + "\033[32m"
            + "PROCESSING GRID CELL ID {} NO. {}/{} CENTROID ({}, {})".format(
                i[2], cell, len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
            )
            + "\033[0m"
        )
    
        # Get the latitude and longitude range of the geopolygon
        lat_range = (geopolygon_gdf.total_bounds[1], geopolygon_gdf.total_bounds[3])
        lon_range = (geopolygon_gdf.total_bounds[0], geopolygon_gdf.total_bounds[2])
    
        # Load Sentinel1 data
        try:
            S1 = load_ard(
                dc=dc,
                products=["s1_rtc"],
                # measurements=["vv", "vh"],
                measurements=["vh"],
                y=lat_range,
                x=lon_range,
                time=timerange,
                output_crs="EPSG:6933",
                resolution=(-20, 20),
                group_by="solar_day",
                dtype="native",
            )
        except Exception as e:
            # Log error aoi centroids and keep looping
            e_log.append([g.x[0], g.y[0], i[2], "P"])
            print(
                "\n\n"
                + "\033[31m"
                + "ERROR PROCESSING GRID CELL {}/{} CENTROID ({}, {}). LOGGED CENTROID INFO in e_log".format(
                    i[2], len(aoi_m), round(g.y[0], 5), round(g.x[0], 5)
                )
                + "\033[0m"
            )
            print("PROCESS ERROR: {}".format(e))
            cell += 1
            continue
    
        # timesteps = [2, 4, 6, 9, 11]
    
        # The lee filter above doesn't handle null values
        # We therefore set null values to 0 before applying the filter
        valid = np.isfinite(S1)
        S1 = S1.where(valid, 0)
    
        # Create a new entry in dataset corresponding to filtered VV and VH data
        S1["filtered_vh"] = S1.vh.groupby("time").apply(lee_filter, size=7)
    
        # Null pixels should remain null
        S1["filtered_vh"] = S1.filtered_vh.where(valid.vh)
    
        # Convert the digital numbers to dB
        S1["filtered_vh"] = 10 * np.log10(S1.filtered_vh)
    
        threshold_vh = th_aoi
    
        S1["water"] = S1_water_classifier(S1.filtered_vh, threshold_vh).s1_water
        FS1 = S1.water
        PRFS1 = S1.water
    
        # Creting outputs
        # Export to raster - upload to g-drive - delete from sandbox

        print('Uploading...')
        # --------------------------------------- preflood ----------------------------------------------
        if i[3] in [None, "P", "U-PRF"]:
            S1_PRF_MD = PRFS1.sel(time=pre_flood, method="nearest").median(dim="time")
            S1_PRF_MN = PRFS1.sel(time=pre_flood, method="nearest").mean(dim="time")
            err = gen_output(S1_PRF_MD, i, geopolygon, cell, aoi_m, 'preflood', 'median')
            if len(err)>0:
                e_log.extend(err)
            err = gen_output(S1_PRF_MN, i, geopolygon, cell, aoi_m, 'preflood', 'mean')
            if len(err)>0:
                e_log.extend(err)

    
        # ----------------------------------------- flood ----------------------------------------------
        if i[3] in [None, "P", "U-F"]:
            S1_F_MD = FS1.sel(time=flood, method="nearest").median(dim="time")
            S1_F_MN = FS1.sel(time=flood, method="nearest").mean(dim="time")
            err = gen_output(S1_F_MD, i, geopolygon, cell, aoi_m, 'flood', 'median')
            if len(err)>0:
                e_log.extend(err)
            err = gen_output(S1_F_MN, i, geopolygon, cell, aoi_m, 'flood', 'mean')
            if len(err)>0:
                e_log.extend(err)
            

        # ----------------------------------------- difference ----------------------------------------------
        if i[3] in [None, "P", "U-FE"]:
            S1_FE_MD = S1_F_MD - S1_PRF_MD
            S1_FE_MN = S1_F_MN - S1_PRF_MN
            err = gen_output(S1_FE_MD, i, geopolygon, cell, aoi_m, 'flood_extents', 'median')
            if len(err)>0:
                e_log.extend(err)
            err = gen_output(S1_FE_MN, i, geopolygon, cell, aoi_m, 'flood_extents', 'mean')
            if len(err)>0:
                e_log.extend(err)

        cell += 1
        clear_output()
    
    if len(e_log) == 0:
        print("\n\n" + "\033[32m" + "GRID PROCESSED AND UPLOADED SUCCESSFULLY" + "\033[0m" + "\n\n")

    # return e_log to be run again
    return e_log

In [9]:
# Crete the aoi-mosaic - aoi_m
def gen_aoim(c, b):
    aoi_m = []
    for i in c:
        aoi_m.append(define_area(i[1], i[0], buffer=b))
    # print(c, len(aoi_m))
    e_log = iterate_grid(aoi_m, c)

    # return e_log to be run again
    return e_log

In [10]:
# Visualize input file
def view_input(gdf_list, grid_c):
    print('Visualizing data...')
    p = gdf_list[0].dissolve()
    center = p.centroid
    map = folium.Map(location=[center.y, center.x], tiles="CartoDB Positron")
    
    for gdf in gdf_list:
        folium.GeoJson(gdf, name="{}".format(gdf)).add_to(map)

    for c in grid_c:
        folium.Marker(
            location=[c[1], c[0]],
            popup=f"Centroid: {c[1]}, {c[0]}",
            icon= folium.DivIcon(
                icon_size=(10, 10),
                icon_anchor=(0,0),
                html='<div style="font-size: 10pt">{}</div>'.format(c[2]),
                )
        ).add_to(map)
    
    bounds = gdf_list[0].total_bounds.tolist()
    map.fit_bounds([bounds[:2][::-1], bounds[2:][::-1]])
    display(map)

In [11]:
# Create grid
from shapely import intersection as intersect
def create_grid(adm0, size):
    bounds = adm0.bounds
    minx = bounds.minx[0] #only 1 feature at the 0th index
    miny = bounds.miny[0]
    maxx = bounds.maxx[0]
    maxy = bounds.maxy[0]

    grid = gpd.GeoDataFrame()
    for x0 in np.arange(minx, maxx, size):
        for y0 in np.arange(miny, maxy, size):
            x1 = x0 + size
            y1 = y0 + size
            d = {'geometry': [Polygon([(x0, y0), (x1, y0), (x1, y1), (x0, y1)])]}
            cell = gpd.GeoDataFrame(d, crs="EPSG:4326")
            flag = adm0.intersection(cell)
            if flag[0].is_empty == False:
                grid = pd.concat([grid, cell])
                
    return grid

In [12]:
# Check CRS and convert to 4326 if required
def crs_check(shp):
    if shp.crs != "EPSG:4326":
        print("Added ADM0 CRS is {}. Converting to EPSG:4326...".format(shp.crs))
        shp = shp.to_crs("EPSG:4326")
        if shp.crs == "EPSG:4326":
            print("Done")

    return shp

### Upload Gridded Vector File

In [13]:
# Load file from sandbox disc. file should be present in 'input' folder
# grid = gpd.read_file("input/Lake Chad.geojson")
# grid = gpd.read_file("input/TCD_55KM_4CTEST.geojson")
# grid = gpd.read_file("input/TCD_55KM_BASE.geojson")
# grid = gpd.read_file("input/TCD_55KM_ERR.geojson")

shp = "input/CMR_ADM0_20KM-B_3857.geojson"
adm0 = gpd.read_file(shp)
adm0 = adm0.dissolve()
adm0 = crs_check(adm0)
size = 0.5  # Grid cell size 0.5 ~ 55KM
buffer = size/2
grid = create_grid(adm0, size)

# Calculate centroids and store in centroid list c[].
c = []
g = grid.centroid

cell_id = 1
for i in g:
    c.append(
        [round(i.x, 5), round(i.y, 5), cell_id, None]
    )  # The array c[] has four values: x, y, cell_id and None. None will store the "P" or "U" error value
    cell_id += 1

# Splice c to test with a few cells
# c = c[:10]

view_input([grid, adm0], c)

Added ADM0 CRS is EPSG:3857. Converting to EPSG:4326...
Done
Visualizing data...


In [36]:
def check_inp(x):
    if x not in ['y', 'n']:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif x == 'n':
        raise RuntimeError("Excecution terminated. Make necessary changes before running again")

def exec_checks():
    inst = """
Before running the execution, ensure all requirements have been met:
1. Create appropriate folders in Google Drive and add their Folder IDs here
2. Verify output folder in Jupyter Notebook (Sandbox Env) and sub folders (flood, preflood, postflood and flood_extents)
3. Check input shapefile
4. Check grid size
    
    """
    print(inst)
    
    x = input("Folder IDs verified? (y/n):")
    check_inp(x)
    
              
    x = input("Sandbox output folders created? (y/n):")
    check_inp(x)
              
    x = input("Input shapefile/geojson verified? (y/n):")
    check_inp(x)
              
    input("Grid (size) verified? (y/n):")
    check_inp(x)
    
    z = input("\nBegin execution for entire input shapefile/geojson? (y/n):")
    if z not in ['y', 'n']:
        raise ValueError("Invalid input, must be 'y' or 'n'")
    elif z == 'n':
        raise RuntimeError("Excecution terminated. Make necessary changes before running again")
    elif z == 'y':
        print("Starting execution...")
        # get e_log with centroids, cell_id and error message
        # Calling gen_aoim will run the entire Application
        e_log = gen_aoim(c, buffer)
        print(len(e_log))
             


### Check and Run Application

In [37]:
# Calls the checklist function
exec_checks()


Before running the execution, ensure all requirements have been met:
1. Create appropriate folders in Google Drive and add their Folder IDs here
2. Verify output folder in Jupyter Notebook (Sandbox Env) and sub folders (flood, preflood, postflood and flood_extents)
3. Check input shapefile
4. Check grid size
    
    


Folder IDs verified? (y/n): y
Sandbox output folders created? (y/n): y
Input shapefile/geojson verified? (y/n): y
Grid (size) verified? (y/n): y

Begin execution for entire input shapefile/geojson? (y/n):
 y


Starting execution...


In [ ]:
# Write error log to disc
e_log = np.array(e_log)
with open('error_centroids.json', 'w') as filehandle:
    json.dump(e_log.tolist(), filehandle)

#read error log from disk
with open('error_centroids.json') as f:
    e_log = json.load(f)
for e in e_log:
    e[0] = float(e[0])
    e[1] = float(e[1])
    e[2] = int(e[2])

try:
    gd.upload_files(["error_centroids.json"], ERR_FOLDER_ID, False)
except Exception as e:
    print("FAILED TO UPLOAD ERROR LOG FILE REASON:{}".format(e))

e_log

In [ ]:
# Visualize Error Cells
e_grid = gpd.GeoDataFrame()
for e in e_log:
    point = Point(e[0], e[1]) #This takes x first and then y
    gdf = gpd.GeoDataFrame(geometry=[point])
    buffer = 0.25
    cell = gpd.GeoDataFrame()
    cell['geometry'] = gdf.buffer(buffer, cap_style='square')
    e_grid = pd.concat([e_grid, cell])
e_grid = e_grid.set_crs('epsg:4326') #e_grid with same cell size as main grid

# e_grid_aoi = e_grid.dissolve()
# e_grid_fine = create_grid(e_grid_aoi, size) # if require to change the size and make it finer

view_input([e_grid], e_log)
    

In [ ]:
# Re-run application for cells logged in e_log
if len(e_log)>0:
    e_log = gen_aoim(e_log, size/2)